## Algoritmo de Teleporte Quântico
### Referencias:

https://learn.qiskit.org/course/ch-algorithms/quantum-teleportation  
https://quantum-computing.ibm.com/lab/docs/iql/manage/systems/dynamic-circuits/Teleportation


In [66]:
# Fazendo as importações necessárias
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import IBMQ, Aer, transpile
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.extensions import Initialize
from qiskit.result import marginal_counts
from qiskit.quantum_info import random_statevector

Alice quer enviar a Bob um estado de um qubit. Pelo teorema da não-clonagem, não é possível criar uma cópia de um estado quântico, independente do método pensado. Então ela precisa de certa forma transferir as informações de seu estado para Bob, o que envolve modificar seu estado. Como eles estão fisicamente distantes, é preciso um par entrelaçado de qubits para a comunicação e um qubit que tem o estado que Alice quer transferir.

In [67]:
qr = QuantumRegister(3, name="q")  
teleportation_circuit = QuantumCircuit(qr)

Para garantir a segurança, um terceiro sujeito deve criar o par entrelaçado (Estado de Bell) e entregar um qubit para Alice e um para Bob. Então o que ela fizer de um lado irá afetar do outro.

In [68]:
def create_bell_pair(qc, a, b):
    """Creates a bell pair in qc using qubits a & b"""
    qc.h(a) # Put qubit a into state |+>
    qc.cx(a,b) # CNOT with a as control and b as target
    qc.barrier()

In [69]:
## STEP 1
# In our case, Telamon entangles qubits q1 and q2
# Let's apply this to our circuit:
create_bell_pair(teleportation_circuit, 1, 2)
# And view the circuit so far:
teleportation_circuit.draw()

░ 
q_0: ───────────░─
     ┌───┐      ░ 
q_1: ┤ H ├──■───░─
     └───┘┌─┴─┐ ░ 
q_2: ─────┤ X ├─░─
          └───┘ ░

Alice então executa uma medida de Bell, que consiste em realizar as operações inversas da criação do Estado de Bell seguido de duas medidas, armazenando cada uuma em um registrador clássico. Os resultados clássicos são lidos e enviados clássicamente para Bob.

In [72]:
def medida_Bell(qc, psi, a, b):
    qc.cx(psi, a)
    qc.h(psi)
    qc.barrier()
    qc.measure(psi,0)
    qc.measure(a,1)

In [73]:
#criação dos registradores clássicos
crz = ClassicalRegister(1, name="crz")
crx = ClassicalRegister(1, name="crx")
#teleportation_circuit = QuantumCircuit(qr, crz, crx)
teleportation_circuit.add_register(crz, crx)

In [74]:
medida_Bell(teleportation_circuit, 0, 1, 2)
teleportation_circuit.draw()

░      ┌───┐ ░ ┌─┐   
  q_0: ───────────░───■──┤ H ├─░─┤M├───
       ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐
  q_1: ┤ H ├──■───░─┤ X ├──────░──╫─┤M├
       └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘
  q_2: ─────┤ X ├─░────────────░──╫──╫─
            └───┘ ░            ░  ║  ║ 
crz: 1/═══════════════════════════╩══╬═
                                  0  ║ 
crx: 1/══════════════════════════════╩═
                                     0

Bob deve então executar algumas operações em seu qubit entrelaçado que dependem do resultado que ele recebeu classicamente.

In [80]:
# This function takes a QuantumCircuit (qc), integer (qubit)
# and ClassicalRegisters (crz & crx) to decide which gates to apply
def bob_gates(qc, qubit, crz, crx):
    qc.barrier()
    # Here we use c_if to control our gates with a classical
    # bit instead of a qubit
    qc.x(qubit).c_if(crx, 1) # Apply gates if the registers 
    qc.z(qubit).c_if(crz, 1) # are in the state '1'

In [81]:
bob_gates(teleportation_circuit, 2, crz, crx)
teleportation_circuit.draw()

░  ░      ┌───┐ ░ ┌─┐    ░                ░               
  q_0: ───────────░──░───■──┤ H ├─░─┤M├────░────────────────░───────────────
       ┌───┐      ░  ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░                ░               
  q_1: ┤ H ├──■───░──░─┤ X ├──────░──╫─┤M├─░────────────────░───────────────
       └───┘┌─┴─┐ ░  ░ └───┘      ░  ║ └╥┘ ░  ┌───┐  ┌───┐  ░  ┌───┐  ┌───┐ 
  q_2: ─────┤ X ├─░──░────────────░──╫──╫──░──┤ X ├──┤ Z ├──░──┤ X ├──┤ Z ├─
            └───┘ ░  ░            ░  ║  ║  ░  └─╥─┘  └─╥─┘  ░  └─╥─┘  └─╥─┘ 
                                     ║  ║       ║   ┌──╨──┐      ║   ┌──╨──┐
crz: 1/══════════════════════════════╩══╬═══════╬═══╡ 0x1 ╞══════╬═══╡ 0x1 ╞
                                     0  ║    ┌──╨──┐└─────┘   ┌──╨──┐└─────┘
crx: 1/═════════════════════════════════╩════╡ 0x1 ╞══════════╡ 0x1 ╞═══════
                                        0    └─────┘          └─────┘